In [ ]:
## 2. Implementing SINDy from Scratch

# Import libraries
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

# Load the data generated in the previous step
data = np.load('lotka_volterra_data.npz')
t = data['t']
X = data['X']

# Step 1: Compute numerical derivatives
# Using a simple finite difference for demonstration
dt = t[1] - t[0]
X_dot = np.gradient(X, axis=0) / dt

# Step 2: Build the library of candidate functions
def build_library(X, poly_degree=2):
    """
    Builds a polynomial library of candidate functions.
    e.g., for X = [x, y], a degree-2 library would be [1, x, y, x^2, xy, y^2]
    """
    n_samples, n_vars = X.shape
    poly_features = []
    
    # Simple polynomial expansion
    for i in range(n_samples):
        features_i = [1]
        for v in range(n_vars):
            # Add linear terms
            features_i.append(X[i, v])
            # Add squared terms
            for p in range(v, n_vars):
                features_i.append(X[i, v] * X[i, p])
        poly_features.append(features_i)

    return np.array(poly_features)

Theta = build_library(X)

# Step 3: Use sparse regression (e.g., L1 regularization)
# This is a simplified implementation of a sparse regression algorithm
def sparsify_dynamics(Theta, X_dot, lam=0.02):
    """
    Finds a sparse solution to Theta * Xi = X_dot using STRidge.
    """
    n_vars = X_dot.shape[1]
    Xi = np.linalg.lstsq(Theta, X_dot, rcond=None)[0] # Initial guess

    for _ in range(10): # Iterative thresholding
        small_coeffs = np.abs(Xi) < lam
        Xi[small_coeffs] = 0
        for i in range(n_vars):
            big_coeffs_idx = np.abs(Xi[:, i]) >= lam
            if np.sum(big_coeffs_idx) > 0:
                Xi[big_coeffs_idx, i] = np.linalg.lstsq(
                    Theta[:, big_coeffs_idx],
                    X_dot[:, i],
                    rcond=None
                )[0]
    return Xi

# Run the sparse regression
Xi = sparsify_dynamics(Theta, X_dot, lam=0.1)

# Step 4: Interpret the results
# The `Xi` matrix contains the coefficients for each term.
print("Discovered Coefficients (Xi matrix):\n", np.round(Xi, 2))

# Let's map coefficients to equations
feature_names = ['1', 'x', 'y', 'x^2', 'xy', 'y^2']
print("\nDiscovered Equations:")
for i, var_name in enumerate(['dx/dt', 'dy/dt']):
    equation = f"{var_name} = "
    terms = []
    for j, coeff in enumerate(Xi[:, i]):
        if np.abs(coeff) > 1e-4:
            terms.append(f"{coeff:.2f} * {feature_names[j]}")
    equation += " + ".join(terms)
    print(equation)